<a href="https://colab.research.google.com/github/AmirHDehghan/Projects/blob/main/Emotional_Attention/Augmentation/Emotion_EEG_Data_Augmentator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
#from dcgan import Discriminator, Generator, weights_init
#from preprocessing import Dataset
import torch.autograd as autograd

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.io as sio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_sess1 = sio.loadmat("/content/drive/MyDrive/Emotion/features/1_20160518.mat")
df_sess2 = sio.loadmat("/content/drive/MyDrive/Emotion/features/1_20161126.mat")
df_sess3 = sio.loadmat("/content/drive/MyDrive/Emotion/features/1_20161125.mat")
#df_sess1 = pd.DataFrame.from_dict(df_sess1)
nu = ["__header__", "__version__", "__globals__"]
for key in nu:
  df_sess1.pop(key)
  df_sess2.pop(key)
  df_sess3.pop(key)
print(len(df_sess1.keys())/4)
print(len(df_sess2.keys())/4)
print(len(df_sess3.keys())/4)

24.0
24.0
24.0


In [ ]:
df_sess2.keys()

dict_keys(['de_movingAve1', 'de_LDS1', 'psd_movingAve1', 'psd_LDS1', 'de_movingAve2', 'de_LDS2', 'psd_movingAve2', 'psd_LDS2', 'de_movingAve3', 'de_LDS3', 'psd_movingAve3', 'psd_LDS3', 'de_movingAve4', 'de_LDS4', 'psd_movingAve4', 'psd_LDS4', 'de_movingAve5', 'de_LDS5', 'psd_movingAve5', 'psd_LDS5', 'de_movingAve6', 'de_LDS6', 'psd_movingAve6', 'psd_LDS6', 'de_movingAve7', 'de_LDS7', 'psd_movingAve7', 'psd_LDS7', 'de_movingAve8', 'de_LDS8', 'psd_movingAve8', 'psd_LDS8', 'de_movingAve9', 'de_LDS9', 'psd_movingAve9', 'psd_LDS9', 'de_movingAve10', 'de_LDS10', 'psd_movingAve10', 'psd_LDS10', 'de_movingAve11', 'de_LDS11', 'psd_movingAve11', 'psd_LDS11', 'de_movingAve12', 'de_LDS12', 'psd_movingAve12', 'psd_LDS12', 'de_movingAve13', 'de_LDS13', 'psd_movingAve13', 'psd_LDS13', 'de_movingAve14', 'de_LDS14', 'psd_movingAve14', 'psd_LDS14', 'de_movingAve15', 'de_LDS15', 'psd_movingAve15', 'psd_LDS15', 'de_movingAve16', 'de_LDS16', 'psd_movingAve16', 'psd_LDS16', 'de_movingAve17', 'de_LDS17', 'ps

In [ ]:
df_sess_labels = np.load("/content/drive/MyDrive/Emotion/labels.npy")
df_sess_labels_1= df_sess_labels[0]
df_sess_labels_1.shape

(24,)

In [ ]:
experiments = []#dict()

for dicts in (df_sess1, df_sess2, df_sess3):
  for keyss, valuess in dicts.items():
    if "de_LDS" in keyss:
      #exp_inx = int(keyss[6:])
      experiments.append(valuess)

experiments = np.concatenate(experiments, axis = 1).transpose((1,0,2))#.reshape((2517, 130))

In [ ]:
experiments.shape[0]

2505

In [ ]:
all_labels = []

for dicts in (df_sess1, df_sess2, df_sess3):
  df_s = len(dicts.keys()) // 4
  for keyss, valuess in dicts.items():
    if "de_LDS" in keyss:
      df_s = df_s -1
      for la in range(valuess.shape[1]):
        all_labels.append(df_sess_labels_1[df_s])
      

In [ ]:
experiments.shape

(2505, 62, 5)

In [ ]:
experiments = experiments.reshape((2505, 62*5))

CLASSIFIER:

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#nsamp, nw, nc = all.shape
x = experiments #.reshape((nsamp, nw*nc))
y = all_labels

#accss = []

#for iiii in range(1, 100):
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = .15)
model_svm = SVC(C=1 , kernel= "rbf", probability= True)
model_svm.fit(xtrain, ytrain)

yhat = model_svm.predict(xtest)
acc_svm = accuracy_score(ytest, yhat)
#accss.append(acc_svm)

print(acc_svm)
#plt.errorbar(np.arange(100), accss)

0.6436170212765957


In [ ]:
x = x.transpose((1,0))
x.shape

(310, 2505)

Model

In [ ]:
beta1 = 0
beta2 = 0.9
p_coeff = 10
n_critic = 5
lr = 1e-5
epoch_num = 64
batch_size = 8
nz = 100  # length of noise
ngpu = 0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            # input 310
            nn.Conv1d(1, 64, kernel_size=8, stride=2, padding=1, bias=False), 
            nn.LeakyReLU(0.2, inplace=True),
            # state size 153
            nn.Conv1d(64, 128, kernel_size=8, stride=2, padding=1, bias=False), 
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2, inplace=True),
            # state size 74
            nn.Conv1d(128, 256, kernel_size=8, stride=2, padding=0, bias=False),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2, inplace=True),
            # state size 34
            nn.Conv1d(256, 512, kernel_size=8, stride=2, padding=0, bias=False),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
            # state size 13
            nn.Conv1d(512, 1, kernel_size=13, stride=1, padding=0, bias=False),
        )

    def forward(self, x, y=None):
        x = self.main(x)
        return x#, print(x.shape, "Dis")



In [ ]:
class Generator(nn.Module):
    def __init__(self, nz):
        super().__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose1d(nz, 512, 13, 1, 0, bias=False),  #13
            nn.BatchNorm1d(512),
            nn.ReLU(True),

            nn.ConvTranspose1d(512, 256, 14, 1, 0, bias=False),  #26
            nn.BatchNorm1d(256),
            nn.ReLU(True),

            nn.ConvTranspose1d(256, 128, 4, 2, 0, bias=False),  #54
            nn.BatchNorm1d(128),
            nn.ReLU(True),

            nn.ConvTranspose1d(128, 64, 4, 2, 0, bias=False),   #110
            nn.BatchNorm1d(64),
            nn.ReLU(True),

            nn.ConvTranspose1d(64, 1, 3, 3, 10, bias=False),   #310
            nn.Tanh()
        )

    def forward(self, x):
        x = self.main(x)
        return x#, print(x.shape, "Gen")

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
tsx = torch.Tensor(experiments)
tsy = torch.Tensor(all_labels)


In [ ]:
def main():
    # load training data
    trainset = TensorDataset(tsx, tsy)

    trainloader = DataLoader(
        trainset, batch_size=batch_size, shuffle=True
    )

    # init netD and netG
    netD = Discriminator().to(device)
    netD.apply(weights_init)

    netG = Generator(nz).to(device)
    netG.apply(weights_init)

    # used for visualizing training process
    fixed_noise = torch.randn(16, nz, 1, device=device)

    # optimizers
    # optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, beta2))
    # optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, beta2))
    optimizerD = optim.RMSprop(netD.parameters(), lr=lr)
    optimizerG = optim.RMSprop(netG.parameters(), lr=lr)

    for epoch in range(epoch_num):
        for step, (data, _) in enumerate(trainloader):
            # training netD
            #print(len(data[7]))
            data = torch.unsqueeze(data, 1)
            real_cpu = data.to(device)
            b_size = real_cpu.size(0)
            #print(real_cpu.shape, "real_cpu")
            netD.zero_grad()

            noise = torch.randn(b_size, nz, 1, device=device)
            fake = netG(noise)
            #print(type(fake))
            # gradient penalty
            eps = torch.Tensor(b_size, 1, 1).uniform_(0, 1)
            #print(eps.shape, "eps")
            x_p = eps * data + (1 - eps) * fake
            #print(x_p.shape,"xp")
            grad = autograd.grad(netD(x_p).mean(), x_p, create_graph=True, retain_graph=True)[0].view(b_size, -1)
            grad_norm = torch.norm(grad, 2, 1)
            grad_penalty = p_coeff * torch.pow(grad_norm - 1, 2)

            loss_D = torch.mean(netD(fake) - netD(real_cpu))
            loss_D.backward()
            optimizerD.step()

            for p in netD.parameters():
                p.data.clamp_(-0.01, 0.01)

            if step % n_critic == 0:
                # training netG
                noise = torch.randn(b_size, nz, 1, device=device)

                netG.zero_grad()
                fake = netG(noise)
                loss_G = -torch.mean(netD(fake))

                netD.zero_grad()
                netG.zero_grad()
                loss_G.backward()
                optimizerG.step()

            if step % 5 == 0:
                print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f'
                      % (epoch, epoch_num, step, len(trainloader), loss_D.item(), loss_G.item()))

        # save training process
        with torch.no_grad():
            fake = netG(fixed_noise).detach().cpu()
            f, a = plt.subplots(4, 4, figsize=(8, 8))
            for i in range(4):
                for j in range(4):
                    a[i][j].plot(fake[i * 4 + j].view(-1))
                    a[i][j].set_xticks(())
                    a[i][j].set_yticks(())
            plt.savefig('/content/drive/MyDrive/Emotion/img/wgan_gp_epoch_%d.png' % epoch)
            plt.close()
    # save model
    torch.save(netG, '/content/drive/MyDrive/Emotion/nets/wgan_gp_netG.pkl')
    torch.save(netD, '/content/drive/MyDrive/Emotion/nets/wgan_gp_netD.pkl')


if __name__ == '__main__':
    main()

[0/64][0/315]	Loss_D: -0.0604	Loss_G: -0.0000
[0/64][5/315]	Loss_D: -0.0057	Loss_G: 0.0000
[0/64][10/315]	Loss_D: -0.0075	Loss_G: 0.0000
[0/64][15/315]	Loss_D: -0.0085	Loss_G: 0.0002
[0/64][20/315]	Loss_D: -0.0103	Loss_G: 0.0003
[0/64][25/315]	Loss_D: -0.0117	Loss_G: 0.0006
[0/64][30/315]	Loss_D: -0.0126	Loss_G: 0.0007
[0/64][35/315]	Loss_D: -0.0141	Loss_G: 0.0007
[0/64][40/315]	Loss_D: -0.0165	Loss_G: 0.0009
[0/64][45/315]	Loss_D: -0.0173	Loss_G: 0.0010
[0/64][50/315]	Loss_D: -0.0175	Loss_G: 0.0011
[0/64][55/315]	Loss_D: -0.0195	Loss_G: 0.0014
[0/64][60/315]	Loss_D: -0.0215	Loss_G: 0.0011
[0/64][65/315]	Loss_D: -0.0218	Loss_G: 0.0014
[0/64][70/315]	Loss_D: -0.0238	Loss_G: 0.0014
[0/64][75/315]	Loss_D: -0.0249	Loss_G: 0.0014
[0/64][80/315]	Loss_D: -0.0255	Loss_G: 0.0014
[0/64][85/315]	Loss_D: -0.0270	Loss_G: 0.0017
[0/64][90/315]	Loss_D: -0.0281	Loss_G: 0.0014
[0/64][95/315]	Loss_D: -0.0293	Loss_G: 0.0015
[0/64][100/315]	Loss_D: -0.0314	Loss_G: 0.0015
[0/64][105/315]	Loss_D: -0.0316	Lo

In [ ]:
new_data = []

netG_model = torch.load('/content/drive/MyDrive/Emotion/nets/wgan_gp_netG.pkl')
for u in range(0, 15000):
  noise = torch.randn(1, nz, 1, device=device)
  nxx = netG_model(noise).to(device).detach().numpy()
  new_data.append(nxx)

In [ ]:
new_data = np.array(new_data)
new_data = new_data.squeeze((1)).squeeze((1))
new_data.shape

(15000, 310)

In [ ]:
new_data_label_p = model_svm.predict_proba(new_data)
new_data_label = model_svm.predict(new_data).reshape((15000,1))

In [ ]:
new_data_label.shape

(15000, 1)

In [ ]:
df_new_labels_p = pd.DataFrame(new_data_label_p, columns= ["Label p: 0", "Label p: 1", "Label p: 2", "Label p: 3"])
df_new_labels = pd.DataFrame(new_data_label, columns= ["Label"])

In [ ]:
df_labs = pd.concat([df_new_labels, df_new_labels_p], axis = 1)
df_labs.head()

,Label,Label p: 0,Label p: 1,Label p: 2,Label p: 3
0,1,0.397507,0.406262,0.106369,0.089862
1,1,0.397507,0.406262,0.106369,0.089862
2,1,0.397507,0.406262,0.106369,0.089862
3,1,0.397507,0.406262,0.106369,0.089862
4,1,0.397507,0.406262,0.106369,0.089862


In [ ]:
df_labs["Label"].mean

<bound method Series.mean of 0        1
1        1
2        1
3        1
4        1
        ..
14995    1
14996    1
14997    1
14998    1
14999    1
Name: Label, Length: 15000, dtype: int32>

./Emotional_Attention/Augmentation/Emotion_EEG_Data_Augmentator.ipynb